# 🚀 Day 3: SAXPY & BLAS Level-1 Operations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-03/day-3-saxpy-blas.ipynb)

---

## 🎯 The Challenge

*Why do NVIDIA, AMD, and Intel all spend millions optimizing a function that just computes `y = a*x + y`? What makes SAXPY the universal GPU benchmark?*

The answer lies in understanding **memory bandwidth** — the true limiting factor for most GPU operations. Today we learn to think like library engineers and benchmark like hardware designers.

---

## 📚 Learning Objectives

By the end of this session, you will be able to:

| Objective | Skill Level |
|-----------|-------------|
| Understand BLAS levels and standard operations | Understand |
| Implement SAXPY, DOT, SCAL, and AXPY operations | Apply |
| Calculate theoretical memory bandwidth | Analyze |
| Evaluate when to write custom kernels vs. use libraries | Evaluate |

---

## 🗺️ Session Roadmap

| Part | Topic | Duration |
|------|-------|----------|
| 1 | Introduction to BLAS | 10 min |
| 2 | SAXPY - The Benchmark | 15 min |
| 3 | Other BLAS Level-1 Operations | 15 min |
| 4 | Bandwidth Analysis | 10 min |
| 5 | Exercises | 10 min |

> **Primary Focus:** CUDA C++ code examples first, Python/Numba backup for interactive testing

---

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math
import time

print(f"\nCUDA available: {cuda.is_available()}")
if cuda.is_available():
    device = cuda.get_current_device()
    print(f"Device: {device.name}")

---

## Part 1: Introduction to BLAS

> 💡 **Concept Card: BLAS as the Gold Standard**
> 
> ```
> 🏆 BLAS - THE BENCHMARK FOR NUMERICAL COMPUTING
> ═══════════════════════════════════════════════════════
> 
>   BLAS = Basic Linear Algebra Subprograms
>   
>   THE GOLD STANDARD because:
>   ✓ Defined in 1979, battle-tested for 45+ years
>   ✓ Every vendor optimizes their implementation
>   ✓ Portable interface, optimized internals
>   ✓ Basis for LAPACK, NumPy, PyTorch, TensorFlow
>   
>   WHEN TO USE LIBRARIES (cuBLAS):
>   ─────────────────────────────────────────────────
>   ✅ Standard operations (GEMM, GEMV, etc.)
>   ✅ Production code needing reliability
>   ✅ When vendor-optimized is faster than DIY
>   
>   WHEN TO WRITE CUSTOM KERNELS:
>   ─────────────────────────────────────────────────
>   ✅ Fusion opportunities (combine ops)
>   ✅ Non-standard operations
>   ✅ Learning and understanding
>   ✅ When library overhead matters
>   
>   TODAY: We learn BOTH to make informed decisions!
>   
> ═══════════════════════════════════════════════════════
> ```
> 
> **The Key Insight:** Understanding BLAS teaches you how to think about performance — even if you end up using the library!

### BLAS Levels

```
BLAS Levels:
━━━━━━━━━━━━
Level 1: Vector-Vector operations  O(n)
  • AXPY:  y = αx + y
  • DOT:   α = x·y
  • SCAL:  x = αx
  • NRM2:  α = ||x||₂

Level 2: Matrix-Vector operations  O(n²)
  • GEMV:  y = αAx + βy
  • SYMV:  y = αAx + βy (A symmetric)

Level 3: Matrix-Matrix operations  O(n³)
  • GEMM:  C = αAB + βC
  • SYMM:  C = αAB + βC (A or B symmetric)
```

### Naming Convention

```
S = Single precision (float32)
D = Double precision (float64)
C = Complex single
Z = Complex double

Examples:
  SAXPY = Single-precision A*X Plus Y
  DGEMM = Double-precision GEneral Matrix Multiply
```

---

## Part 2: SAXPY - The Hello World of GPU Computing

### The Operation

$$y_i = \alpha \cdot x_i + y_i$$

> 💡 **Concept Card: Why SAXPY is THE Benchmark**
> 
> ```
> 📊 SAXPY: THE MEMORY BANDWIDTH BENCHMARK
> ═══════════════════════════════════════════════════════
> 
>   y = α × x + y  (for each element)
>   
>   MEMORY TRAFFIC ANALYSIS:
>   ─────────────────────────────────────────────────
>   • Read x[i]:   4 bytes (float32)
>   • Read y[i]:   4 bytes (float32)
>   • Write y[i]:  4 bytes (float32)
>   ─────────────────────────────────────────────────
>   Total:         12 bytes per element
>   
>   COMPUTE:
>   ─────────────────────────────────────────────────
>   • 1 multiply (α × x[i])
>   • 1 add (result + y[i])
>   ─────────────────────────────────────────────────
>   Total:         2 FLOPs per element
>   
>   ARITHMETIC INTENSITY:
>   ─────────────────────────────────────────────────
>   2 FLOPs ÷ 12 bytes = 0.17 FLOPs/byte
>   
>   ⚠️ MEMORY-BOUND! (GPUs can do 100+ FLOPs/byte)
>   
>   WHY IT'S THE BENCHMARK:
>   → Measures raw memory bandwidth
>   → Cannot hide behind compute
>   → Honest performance indicator
>   
> ═══════════════════════════════════════════════════════
> ```
> 
> **The Takeaway:** SAXPY performance = your GPU's memory system performance!

### 🔷 CUDA C++ Implementation (Primary)

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
%%writefile saxpy.cu
// saxpy.cu - The classic GPU benchmark
#include <stdio.h>
#include <cuda_runtime.h>

// SAXPY: y = alpha * x + y
__global__ void saxpy(float alpha, const float* x, float* y, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    for (int i = tid; i < n; i += stride) {
        y[i] = alpha * x[i] + y[i];
    }
}

int main() {
    int n = 1000000;
    float alpha = 2.0f;
    size_t size = n * sizeof(float);
    
    // Allocate host memory
    float *h_x = (float*)malloc(size);
    float *h_y = (float*)malloc(size);
    
    // Initialize
    for (int i = 0; i < n; i++) {
        h_x[i] = 1.0f;
        h_y[i] = 2.0f;
    }
    
    // Allocate device memory
    float *d_x, *d_y;
    cudaMalloc(&d_x, size);
    cudaMalloc(&d_y, size);
    
    // Copy to device
    cudaMemcpy(d_x, h_x, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_y, h_y, size, cudaMemcpyHostToDevice);
    
    // Launch SAXPY
    int threads = 256;
    int blocks = 256;
    saxpy<<<blocks, threads>>>(alpha, d_x, d_y, n);
    cudaDeviceSynchronize();
    
    // Copy back and verify
    cudaMemcpy(h_y, d_y, size, cudaMemcpyDeviceToHost);
    
    // y[0] should be: 2.0 * 1.0 + 2.0 = 4.0
    printf("y[0] = %f (expected 4.0)\n", h_y[0]);
    
    // Cleanup
    cudaFree(d_x); cudaFree(d_y);
    free(h_x); free(h_y);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o saxpy saxpy.cu
!./saxpy

In [ ]:
%%writefile blas_level1.cu
// blas_level1.cu - Other BLAS Level-1 Operations
#include <stdio.h>
#include <cuda_runtime.h>

// SCAL: x = alpha * x
__global__ void scal(float alpha, float* x, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    for (int i = tid; i < n; i += stride) {
        x[i] = alpha * x[i];
    }
}

// AXPY: y = alpha * x + y (same as SAXPY)
__global__ void axpy(float alpha, const float* x, float* y, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    for (int i = tid; i < n; i += stride) {
        y[i] = alpha * x[i] + y[i];
    }
}

// DOT: result = sum(x[i] * y[i]) - requires reduction!
// See Week 4 for proper implementation
__global__ void dot_partial(const float* x, const float* y, float* partial, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    float sum = 0.0f;
    for (int i = tid; i < n; i += stride) {
        sum += x[i] * y[i];
    }
    partial[tid] = sum;  // Needs reduction to complete
}

int main() {
    int n = 1000000;
    float alpha = 2.0f;
    size_t size = n * sizeof(float);
    
    float *h_x = (float*)malloc(size);
    for (int i = 0; i < n; i++) h_x[i] = 3.0f;
    
    float *d_x;
    cudaMalloc(&d_x, size);
    cudaMemcpy(d_x, h_x, size, cudaMemcpyHostToDevice);
    
    int threads = 256, blocks = 256;
    
    // Test SCAL
    scal<<<blocks, threads>>>(alpha, d_x, n);
    cudaMemcpy(h_x, d_x, size, cudaMemcpyDeviceToHost);
    printf("SCAL: 3.0 * 2.0 = %f (expected 6.0)\n", h_x[0]);
    
    cudaFree(d_x);
    free(h_x);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o blas_level1 blas_level1.cu
!./blas_level1

In [ ]:
# Python equivalent for interactive testing
@cuda.jit
def saxpy(alpha, x, y, n):
    """
    SAXPY: y = alpha * x + y
    Modifies y in-place.
    """
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        y[i] = alpha * x[i] + y[i]

In [ ]:
# Test SAXPY
n = 1_000_000
alpha = 2.0

x = np.random.rand(n).astype(np.float32)
y = np.random.rand(n).astype(np.float32)
y_original = y.copy()

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

blocks, threads = 256, 256
saxpy[blocks, threads](alpha, d_x, d_y, n)

result = d_y.copy_to_host()
expected = alpha * x + y_original

print(f"SAXPY: y = {alpha} * x + y")
print(f"N = {n:,}")
print(f"Correct: {np.allclose(result, expected)}")
print(f"\nSample results (first 5):")
print(f"  x:        {x[:5]}")
print(f"  y (orig): {y_original[:5]}")
print(f"  y (new):  {result[:5]}")

---

## Part 3: Other BLAS Level-1 Operations

Now let's complete our BLAS Level-1 toolkit with SCAL, DOT, and AXPY operations.

> 💡 **Concept Card: BLAS Level-1 Memory Patterns**
> 
> ```
> 📦 BLAS-1 OPERATIONS: Memory Traffic Analysis
> ═══════════════════════════════════════════════════════
> 
>   ┌──────────┬─────────────┬────────────┬────────────┐
>   │ Operation│ Formula     │ Bytes/elem │ FLOPs/elem │
>   ├──────────┼─────────────┼────────────┼────────────┤
>   │ SCAL     │ x = αx      │ 8 (R+W)    │ 1          │
>   │ COPY     │ y = x       │ 8 (R+W)    │ 0          │
>   │ AXPY     │ y = αx + y  │ 12 (2R+W)  │ 2          │
>   │ DOT      │ α = x·y     │ 8 (2R)     │ 2          │
>   │ NRM2     │ α = ‖x‖₂    │ 4 (1R)     │ 2          │
>   └──────────┴─────────────┴────────────┴────────────┘
>   
>   ALL ARE MEMORY-BOUND! (Arithmetic intensity < 1)
>   
>   ⚠️ DOT and NRM2 require REDUCTION (Week 4 topic)
>   
> ═══════════════════════════════════════════════════════
> ```
> 
> **Notice:** All Level-1 operations are memory-bound. The GPU's compute units are mostly waiting for data!

### 🔷 CUDA C++ Implementation (Primary)

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
def benchmark_saxpy(n, iterations=100):
    """Benchmark SAXPY and calculate effective bandwidth."""
    alpha = 2.0
    x = np.random.rand(n).astype(np.float32)
    y = np.random.rand(n).astype(np.float32)
    
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)
    
    blocks, threads = 256, 256
    
    # Warmup
    saxpy[blocks, threads](alpha, d_x, d_y, n)
    cuda.synchronize()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        saxpy[blocks, threads](alpha, d_x, d_y, n)
    cuda.synchronize()
    elapsed = (time.perf_counter() - start) / iterations
    
    # Calculate bandwidth
    bytes_transferred = n * 12  # 3 arrays × 4 bytes (read x, read y, write y)
    bandwidth_gb_s = (bytes_transferred / elapsed) / 1e9
    
    return elapsed * 1000, bandwidth_gb_s  # ms, GB/s

print(f"{'N':>12} | {'Time (ms)':>10} | {'Bandwidth (GB/s)':>16}")
print("-" * 45)

for n in [100_000, 1_000_000, 10_000_000, 100_000_000]:
    time_ms, bw = benchmark_saxpy(n)
    print(f"{n:>12,} | {time_ms:>10.3f} | {bw:>16.1f}")

---

## Part 4: Bandwidth Analysis

Let's measure and understand the actual memory bandwidth we're achieving.

### The Bandwidth Formula

```
Effective Bandwidth = Bytes Transferred / Time

For SAXPY:
  Bandwidth = (3 × N × sizeof(float)) / Time
            = (12 × N) / Time  [bytes/second]
```

### Typical GPU Memory Bandwidths

| GPU | Theoretical Bandwidth | Achievable (~80%) |
|-----|----------------------|-------------------|
| RTX 3090 | 936 GB/s | ~750 GB/s |
| A100 | 2,039 GB/s | ~1,600 GB/s |
| RTX 4090 | 1,008 GB/s | ~800 GB/s |
| V100 | 900 GB/s | ~720 GB/s |

> **Goal:** Get as close to theoretical bandwidth as possible!

---

## Part 5: Exercises

Now it's your turn to implement BLAS-like operations:

---

## Part 4: Other BLAS Level-1 Operations

In [ ]:
# SCAL: x = alpha * x
@cuda.jit
def sscal(alpha, x, n):
    """Scale vector: x = alpha * x"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        x[i] = alpha * x[i]

# COPY: y = x
@cuda.jit
def scopy(x, y, n):
    """Copy vector: y = x"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        y[i] = x[i]

# SWAP: swap x and y
@cuda.jit
def sswap(x, y, n):
    """Swap vectors: x, y = y, x"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        temp = x[i]
        x[i] = y[i]
        y[i] = temp

In [ ]:
# DOT product (partial - needs reduction for full result)
@cuda.jit
def sdot_partial(x, y, partial_sums, n):
    """
    Partial dot product: each thread computes partial sum.
    Full reduction needed to get final result.
    """
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    local_sum = 0.0
    for i in range(tid, n, stride):
        local_sum += x[i] * y[i]
    
    partial_sums[tid] = local_sum

def sdot(x, y):
    """Complete dot product implementation."""
    n = len(x)
    blocks, threads = 256, 256
    total_threads = blocks * threads
    
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)
    d_partial = cuda.device_array(total_threads, dtype=np.float32)
    
    sdot_partial[blocks, threads](d_x, d_y, d_partial, n)
    
    # Final reduction on CPU (we'll learn GPU reduction in Week 4)
    partial = d_partial.copy_to_host()
    return partial.sum()

In [ ]:
# Test BLAS operations
n = 100_000
x = np.random.rand(n).astype(np.float32)
y = np.random.rand(n).astype(np.float32)

blocks, threads = 256, 256

# Test SCAL
d_x = cuda.to_device(x.copy())
sscal[blocks, threads](2.0, d_x, n)
result = d_x.copy_to_host()
print(f"SCAL (x *= 2): {'✓' if np.allclose(result, x * 2) else '✗'}")

# Test COPY
d_x = cuda.to_device(x)
d_y = cuda.device_array(n, dtype=np.float32)
scopy[blocks, threads](d_x, d_y, n)
result = d_y.copy_to_host()
print(f"COPY (y = x): {'✓' if np.allclose(result, x) else '✗'}")

# Test SWAP
d_x = cuda.to_device(x.copy())
d_y = cuda.to_device(y.copy())
sswap[blocks, threads](d_x, d_y, n)
result_x = d_x.copy_to_host()
result_y = d_y.copy_to_host()
print(f"SWAP: {'✓' if np.allclose(result_x, y) and np.allclose(result_y, x) else '✗'}")

# Test DOT
gpu_dot = sdot(x, y)
cpu_dot = np.dot(x, y)
print(f"DOT: {'✓' if np.isclose(gpu_dot, cpu_dot, rtol=1e-4) else '✗'} (GPU: {gpu_dot:.4f}, CPU: {cpu_dot:.4f})")

---

## Part 5: NRM2 - Vector Norm

In [ ]:
@cuda.jit
def snrm2_partial(x, partial_sums, n):
    """
    Partial L2 norm squared: each thread computes sum of squares.
    Need to sqrt the final sum.
    """
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    local_sum = 0.0
    for i in range(tid, n, stride):
        local_sum += x[i] * x[i]
    
    partial_sums[tid] = local_sum

def snrm2(x):
    """Complete L2 norm: ||x||_2 = sqrt(sum(x_i^2))"""
    n = len(x)
    blocks, threads = 256, 256
    total_threads = blocks * threads
    
    d_x = cuda.to_device(x)
    d_partial = cuda.device_array(total_threads, dtype=np.float32)
    
    snrm2_partial[blocks, threads](d_x, d_partial, n)
    
    partial = d_partial.copy_to_host()
    return np.sqrt(partial.sum())

# Test
x = np.random.rand(100_000).astype(np.float32)
gpu_norm = snrm2(x)
cpu_norm = np.linalg.norm(x)

print(f"L2 Norm:")
print(f"  GPU: {gpu_norm:.6f}")
print(f"  CPU: {cpu_norm:.6f}")
print(f"  Match: {'✓' if np.isclose(gpu_norm, cpu_norm, rtol=1e-4) else '✗'}")

---

## Part 6: Extended AXPY Variants

In [ ]:
# AXPBY: y = alpha*x + beta*y (more general)
@cuda.jit
def saxpby(alpha, x, beta, y, n):
    """y = alpha*x + beta*y"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        y[i] = alpha * x[i] + beta * y[i]

# WAXPBY: w = alpha*x + beta*y (output to separate array)
@cuda.jit
def swaxpby(alpha, x, beta, y, w, n):
    """w = alpha*x + beta*y"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        w[i] = alpha * x[i] + beta * y[i]

# Triple AXPY: y = a1*x1 + a2*x2 + a3*x3
@cuda.jit
def saxpy3(a1, x1, a2, x2, a3, x3, y, n):
    """y = a1*x1 + a2*x2 + a3*x3"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        y[i] = a1 * x1[i] + a2 * x2[i] + a3 * x3[i]

In [ ]:
# Test AXPBY
n = 100_000
alpha, beta = 2.0, 0.5
x = np.random.rand(n).astype(np.float32)
y = np.random.rand(n).astype(np.float32)
y_orig = y.copy()

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

saxpby[256, 256](alpha, d_x, beta, d_y, n)
result = d_y.copy_to_host()
expected = alpha * x + beta * y_orig

print(f"AXPBY (y = {alpha}*x + {beta}*y): {'✓' if np.allclose(result, expected) else '✗'}")

---

## Part 7: Building a Vector Library Class

In [ ]:
class CUDAVector:
    """CUDA-accelerated vector operations."""
    
    def __init__(self, blocks=256, threads=256):
        self.blocks = blocks
        self.threads = threads
    
    def _get_device_arrays(self, *arrays):
        """Convert numpy arrays to device arrays if needed."""
        result = []
        for arr in arrays:
            if isinstance(arr, np.ndarray):
                result.append(cuda.to_device(arr))
            else:
                result.append(arr)
        return result
    
    def axpy(self, alpha, x, y):
        """y = alpha*x + y (in-place)"""
        d_x, d_y = self._get_device_arrays(x, y)
        n = len(x)
        saxpy[self.blocks, self.threads](alpha, d_x, d_y, n)
        return d_y
    
    def axpby(self, alpha, x, beta, y):
        """y = alpha*x + beta*y (in-place)"""
        d_x, d_y = self._get_device_arrays(x, y)
        n = len(x)
        saxpby[self.blocks, self.threads](alpha, d_x, beta, d_y, n)
        return d_y
    
    def scal(self, alpha, x):
        """x = alpha*x (in-place)"""
        d_x, = self._get_device_arrays(x)
        n = len(x)
        sscal[self.blocks, self.threads](alpha, d_x, n)
        return d_x
    
    def dot(self, x, y):
        """Return x·y"""
        d_x, d_y = self._get_device_arrays(x, y)
        n = len(x)
        total_threads = self.blocks * self.threads
        d_partial = cuda.device_array(total_threads, dtype=np.float32)
        
        sdot_partial[self.blocks, self.threads](d_x, d_y, d_partial, n)
        return d_partial.copy_to_host().sum()
    
    def nrm2(self, x):
        """Return ||x||_2"""
        d_x, = self._get_device_arrays(x)
        n = len(x)
        total_threads = self.blocks * self.threads
        d_partial = cuda.device_array(total_threads, dtype=np.float32)
        
        snrm2_partial[self.blocks, self.threads](d_x, d_partial, n)
        return np.sqrt(d_partial.copy_to_host().sum())

In [ ]:
# Test the vector library
vec = CUDAVector()

n = 100_000
x = np.random.rand(n).astype(np.float32)
y = np.random.rand(n).astype(np.float32)

# Test all operations
print("CUDAVector Library Tests:")
print("-" * 40)

# DOT
gpu_dot = vec.dot(x, y)
cpu_dot = np.dot(x, y)
print(f"dot(x, y): {'✓' if np.isclose(gpu_dot, cpu_dot, rtol=1e-4) else '✗'}")

# NRM2
gpu_norm = vec.nrm2(x)
cpu_norm = np.linalg.norm(x)
print(f"nrm2(x):   {'✓' if np.isclose(gpu_norm, cpu_norm, rtol=1e-4) else '✗'}")

# SCAL
x_copy = x.copy()
d_x = vec.scal(3.0, x_copy)
result = d_x.copy_to_host()
print(f"scal(3,x): {'✓' if np.allclose(result, x * 3) else '✗'}")

# AXPY
y_copy = y.copy()
d_y = vec.axpy(2.0, x, y_copy)
result = d_y.copy_to_host()
print(f"axpy:      {'✓' if np.allclose(result, 2*x + y) else '✗'}")

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these BLAS-style exercises in CUDA C++.

In [ ]:
%%writefile saxpy_exercises.cu
// saxpy_exercises.cu - BLAS-style exercises
#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: SASUM - Sum of Absolute Values
// =============================================================================

__global__ void sasumPartial(const float* x, float* partialSums, int n) {
    extern __shared__ float sdata[];
    
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load with absolute value
    sdata[tid] = (idx < n) ? fabsf(x[idx]) : 0.0f;
    __syncthreads();
    
    // Parallel reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    // Write result for this block
    if (tid == 0) {
        partialSums[blockIdx.x] = sdata[0];
    }
}

float sasum(const float* d_x, int n) {
    int threads = 256;
    int blocks = (n + threads - 1) / threads;
    
    float* d_partial;
    CUDA_CHECK(cudaMalloc(&d_partial, blocks * sizeof(float)));
    
    sasumPartial<<<blocks, threads, threads * sizeof(float)>>>(d_x, d_partial, n);
    
    // Final reduction on CPU (or recursively on GPU for very large arrays)
    float* h_partial = (float*)malloc(blocks * sizeof(float));
    CUDA_CHECK(cudaMemcpy(h_partial, d_partial, blocks * sizeof(float), cudaMemcpyDeviceToHost));
    
    float sum = 0.0f;
    for (int i = 0; i < blocks; i++) sum += h_partial[i];
    
    free(h_partial);
    cudaFree(d_partial);
    return sum;
}

// =============================================================================
// Exercise 2: ISAMAX - Index of Maximum Absolute Value
// =============================================================================

__global__ void isamaxPartial(const float* x, float* maxVals, int* maxIdx, int n) {
    extern __shared__ char sharedMem[];
    float* svals = (float*)sharedMem;
    int* sidx = (int*)(sharedMem + blockDim.x * sizeof(float));
    
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Initialize
    svals[tid] = (idx < n) ? fabsf(x[idx]) : -1.0f;
    sidx[tid] = idx;
    __syncthreads();
    
    // Parallel reduction keeping track of max and index
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            if (svals[tid + s] > svals[tid]) {
                svals[tid] = svals[tid + s];
                sidx[tid] = sidx[tid + s];
            }
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        maxVals[blockIdx.x] = svals[0];
        maxIdx[blockIdx.x] = sidx[0];
    }
}

int isamax(const float* d_x, int n) {
    int threads = 256;
    int blocks = (n + threads - 1) / threads;
    
    float* d_maxVals;
    int* d_maxIdx;
    CUDA_CHECK(cudaMalloc(&d_maxVals, blocks * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_maxIdx, blocks * sizeof(int)));
    
    size_t sharedSize = threads * (sizeof(float) + sizeof(int));
    isamaxPartial<<<blocks, threads, sharedSize>>>(d_x, d_maxVals, d_maxIdx, n);
    
    float* h_maxVals = (float*)malloc(blocks * sizeof(float));
    int* h_maxIdx = (int*)malloc(blocks * sizeof(int));
    CUDA_CHECK(cudaMemcpy(h_maxVals, d_maxVals, blocks * sizeof(float), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_maxIdx, d_maxIdx, blocks * sizeof(int), cudaMemcpyDeviceToHost));
    
    // Final comparison on CPU
    float maxVal = h_maxVals[0];
    int maxIndex = h_maxIdx[0];
    for (int i = 1; i < blocks; i++) {
        if (h_maxVals[i] > maxVal) {
            maxVal = h_maxVals[i];
            maxIndex = h_maxIdx[i];
        }
    }
    
    free(h_maxVals); free(h_maxIdx);
    cudaFree(d_maxVals); cudaFree(d_maxIdx);
    return maxIndex;
}

// =============================================================================
// Exercise 3: Batch SAXPY (apply to rows of matrices)
// =============================================================================

__global__ void batchSaxpy(float alpha, const float* X, float* Y, int M, int N) {
    // Each thread handles one element
    int row = blockIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < M && col < N) {
        int idx = row * N + col;
        Y[idx] = alpha * X[idx] + Y[idx];
    }
}

// =============================================================================
// Test harness
// =============================================================================

int main() {
    printf("=== SAXPY/BLAS Exercises ===\n\n");
    
    // Exercise 1: SASUM
    printf("Exercise 1: SASUM (Sum of Absolute Values)\n");
    printf("-" "------------------------------------------\n");
    {
        float h_x[] = {-1.0f, 2.0f, -3.0f, 4.0f, -5.0f};
        const int N = 5;
        
        float* d_x;
        CUDA_CHECK(cudaMalloc(&d_x, N * sizeof(float)));
        CUDA_CHECK(cudaMemcpy(d_x, h_x, N * sizeof(float), cudaMemcpyHostToDevice));
        
        float result = sasum(d_x, N);
        printf("Input: [-1, 2, -3, 4, -5]\n");
        printf("SASUM = %.0f (expected: 15)\n\n", result);
        
        cudaFree(d_x);
    }
    
    // Exercise 2: ISAMAX
    printf("Exercise 2: ISAMAX (Index of Max Abs Value)\n");
    printf("-" "-------------------------------------------\n");
    {
        float h_x[] = {1.0f, -5.0f, 3.0f, -2.0f};
        const int N = 4;
        
        float* d_x;
        CUDA_CHECK(cudaMalloc(&d_x, N * sizeof(float)));
        CUDA_CHECK(cudaMemcpy(d_x, h_x, N * sizeof(float), cudaMemcpyHostToDevice));
        
        int result = isamax(d_x, N);
        printf("Input: [1, -5, 3, -2]\n");
        printf("ISAMAX = %d (expected: 1, value=-5, |−5|=5 is max)\n\n", result);
        
        cudaFree(d_x);
    }
    
    // Exercise 3: Batch SAXPY
    printf("Exercise 3: Batch SAXPY\n");
    printf("-" "-----------------------\n");
    {
        const int M = 10, N = 1000;
        float* h_X = (float*)malloc(M * N * sizeof(float));
        float* h_Y = (float*)malloc(M * N * sizeof(float));
        
        for (int i = 0; i < M * N; i++) {
            h_X[i] = 1.0f;
            h_Y[i] = 2.0f;
        }
        
        float *d_X, *d_Y;
        CUDA_CHECK(cudaMalloc(&d_X, M * N * sizeof(float)));
        CUDA_CHECK(cudaMalloc(&d_Y, M * N * sizeof(float)));
        CUDA_CHECK(cudaMemcpy(d_X, h_X, M * N * sizeof(float), cudaMemcpyHostToDevice));
        CUDA_CHECK(cudaMemcpy(d_Y, h_Y, M * N * sizeof(float), cudaMemcpyHostToDevice));
        
        dim3 block(256);
        dim3 grid((N + 255) / 256, M);
        
        batchSaxpy<<<grid, block>>>(3.0f, d_X, d_Y, M, N);
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_Y, d_Y, M * N * sizeof(float), cudaMemcpyDeviceToHost));
        
        printf("Applied Y = 3*X + Y to %d vectors of length %d\n", M, N);
        printf("Sample Y[0][0] = %.0f (expected: 3*1 + 2 = 5)\n", h_Y[0]);
        
        cudaFree(d_X); cudaFree(d_Y);
        free(h_X); free(h_Y);
    }
    
    printf("\n=== All exercises complete! ===\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o saxpy_exercises saxpy_exercises.cu && ./saxpy_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: ASUM - Sum of Absolute Values

In [ ]:
# TODO: Implement SASUM: sum of |x_i|
@cuda.jit
def sasum_partial(x, partial_sums, n):
    """Compute sum of absolute values."""
    # Hint: Use math.fabs(x[i])
    pass

def sasum(x):
    """Return sum(|x_i|)"""
    # Your implementation
    pass

# Test: x = [-1, 2, -3, 4, -5]
# Expected: 1 + 2 + 3 + 4 + 5 = 15

### Exercise 2: IAMAX - Index of Maximum Absolute Value

In [ ]:
# TODO: Find index of max |x_i|
# This is tricky with parallelism - think about how to do it!

# Hint: Each thread finds max in its range, then combine
@cuda.jit
def isamax_partial(x, partial_max_vals, partial_max_idx, n):
    """Each thread finds local max abs value and its index."""
    pass

# Test: x = [1, -5, 3, -2]
# Expected: index 1 (value -5, |−5| = 5 is largest)

### Exercise 3: Batch AXPY

In [ ]:
# TODO: Apply AXPY to multiple vectors at once
# Given: X (M x N matrix), Y (M x N matrix)
# Compute: Y[i] = alpha * X[i] + Y[i] for each row i

@cuda.jit
def batch_saxpy(alpha, X, Y, M, N):
    """
    Apply SAXPY to each row of X and Y.
    X, Y are M×N matrices.
    """
    pass

# Test with 10 vectors of length 1000 each

---

## 📝 Key Takeaways

### Quick Reference Card: BLAS Level-1

```
┌─────────────────────────────────────────────────────────────────┐
│  BLAS LEVEL-1 OPERATIONS                                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  OPERATION TABLE:                                               │
│  ───────────────────────────────────────────────────────────    │
│  │ Op    │ Formula     │ Bytes/elem │ FLOPs/elem │ Notes    │  │
│  │ SCAL  │ x = αx      │ 8 (R+W)    │ 1          │ In-place │  │
│  │ COPY  │ y = x       │ 8 (R+W)    │ 0          │ Pure I/O │  │
│  │ AXPY  │ y = αx + y  │ 12 (2R+W)  │ 2          │ SAXPY    │  │
│  │ DOT   │ α = x·y     │ 8 (2R)     │ 2          │ Reduce   │  │
│  │ NRM2  │ α = ‖x‖₂    │ 4 (1R)     │ 2          │ Reduce   │  │
│                                                                 │
│  BANDWIDTH FORMULA:                                             │
│  ───────────────────────────────────────────────────────────    │
│  Effective BW = (Bytes Transferred) / Time                      │
│                                                                 │
│  For SAXPY: BW = (3 × N × 4 bytes) / Time = 12N / Time          │
│                                                                 │
│  LIBRARY VS CUSTOM:                                             │
│  ───────────────────────────────────────────────────────────    │
│  cuBLAS: Use for standard ops, production code                  │
│  Custom: Use for fusion, learning, non-standard ops             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### ✅ What You Achieved Today

| Skill | Status |
|-------|--------|
| Understand BLAS levels and standard operations | ✅ |
| Implemented SAXPY, SCAL, AXPY | ✅ |
| Calculated theoretical memory bandwidth | ✅ |
| Understand when to use libraries vs custom | ✅ |

### 🧠 Key Insights

1. **BLAS Level-1 is memory-bound**: Low arithmetic intensity means we're limited by memory bandwidth
2. **SAXPY is the benchmark**: It honestly measures memory system performance
3. **Reduction operations need special handling**: DOT and NRM2 require techniques from Week 4
4. **Professional libraries use these patterns**: cuBLAS, MKL, OpenBLAS all follow this standard

---

## 🚀 What's Next?

**Day 4: Kernel Fusion & Optimization** — Combining multiple operations into single kernels for massive performance gains!

| Preview Topic | What You'll Learn |
|---------------|-------------------|
| The fusion problem | Why separate kernels waste bandwidth |
| Fused operations | Combining ops in single kernel |
| Memory traffic analysis | Measuring reduction in traffic |
| Fused activations | Real-world neural network patterns |

> 💡 **Tomorrow's Hook:** What if you could do 4 operations for the memory cost of 1? That's the power of kernel fusion!